In [1]:
# Import libraries
import pandas as pd
import requests
from datetime import date

# Headers for API Request
nba_header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [13]:
# Functions
def get_nba_games_today(nba_header_data):
    
    ### Function to scrape NBA API for today's games
    ## Args: 
    # nba_header_data (dict): headers for NBA API request
    ## Returns (df): dataframe of nba_games_today with today's games
    ###
    
    # Get system date
    today = date.today()
    
    # Paste into url
    url = 'https://stats.nba.com/stats/scoreboardv3?GameDate=' + str(today) + '&LeagueID=00'
    
    # Send request
    r = requests.get(url, headers=nba_header_data)
    
    # Get JSON response
    resp = r.json()
    
    # Set json resp columns to grab
    nba_json_schedule_cols = ['gameId', 'gameEt', 'awayTeam.teamId', 'awayTeam.teamTricode', 'awayTeam.teamName',
                            'homeTeam.teamId', 'homeTeam.teamTricode', 'homeTeam.teamName']

    # Normalize json and select columns
    nba_games_today = pd.json_normalize(resp["scoreboard"]['games'])
    
    # If games
    if len(nba_games_today) > 0:
        nba_games_today = nba_games_today[nba_json_schedule_cols]

        # Create awayteamName and homeTeamName
        nba_games_today['awayTeamName'] = nba_games_today['awayTeam.teamTricode'] + ' ' + nba_games_today['awayTeam.teamName']
        nba_games_today['homeTeamName'] = nba_games_today['homeTeam.teamTricode'] + ' ' + nba_games_today['homeTeam.teamName']

        # Rename some columns
        nba_games_today.rename({'awayTeam.teamId': 'awayTeamId', 'awayTeam.teamTricode': 'awayTeamSlug', 
                                'homeTeam.teamId': 'homeTeamId', 'homeTeam.teamTricode': 'homeTeamSlug'}, axis=1, inplace=True)

        # Set columns to select
        nba_schedule_cols = ['gameId', 'gameEt', 'awayTeamId', 'awayTeamSlug', 'awayTeamName',
                            'homeTeamId', 'homeTeamSlug', 'homeTeamName']

        # Select columns
        nba_games_today = nba_games_today[nba_schedule_cols]

        ## --- INSERT/UPDATE SQL --- ##
        # - Update all other columns where game id = gameId - #
        
        return nba_games_today 
    else:
        print('No Games Found Today')

In [14]:
# Get today's schedule
get_nba_games_today(nba_header_data)

,gameId,gameEt,awayTeamId,awayTeamSlug,awayTeamName,homeTeamId,homeTeamSlug,homeTeamName
0,0022200452,2022-12-19T19:00:00Z,1610612762,UTA,UTA Jazz,1610612739,CLE,CLE Cavaliers
1,0022200453,2022-12-19T19:00:00Z,1610612761,TOR,TOR Raptors,1610612755,PHI,PHI 76ers
2,0022200454,2022-12-19T19:30:00Z,1610612753,ORL,ORL Magic,1610612737,ATL,ATL Hawks
3,0022200455,2022-12-19T20:00:00Z,1610612759,SAS,SAS Spurs,1610612745,HOU,HOU Rockets
4,0022200456,2022-12-19T20:00:00Z,1610612742,DAL,DAL Mavericks,1610612750,MIN,MIN Timberwolves
5,0022200457,2022-12-19T20:00:00Z,1610612749,MIL,MIL Bucks,1610612740,NOP,NOP Pelicans
6,0022200458,2022-12-19T20:00:00Z,1610612757,POR,POR Trail Blazers,1610612760,OKC,OKC Thunder
7,0022200459,2022-12-19T21:00:00Z,1610612747,LAL,LAL Lakers,1610612756,PHX,PHX Suns
8,0022200460,2022-12-19T22:00:00Z,1610612766,CHA,CHA Hornets,1610612758,SAC,SAC Kings
